# Introduction

This notebook demonstrates how to perform structure enumeration using Python Materials Genomics (pymatgen).

Let's start by importing some modules and classes that we will be using.

In [1]:
from pymatgen.core import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.transformations.advanced_transformations import EnumerateStructureTransformation
from pymatgen.io.vasp.sets import batch_write_input, MPRelaxSet

# Preparation

We will first read in the structure from crystallographic information file (CIF) found in ICSD.

In [2]:
structure = Structure.from_file("EntryWithCollCode418490.cif")
print(structure)

Full Formula (Li26.88 P4 S20 Cl4)
Reduced Formula: Li26.88P4S20Cl4
abc   :   9.859000   9.859000   9.859000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (76)
  #  SP              a        b        c
---  --------  -------  -------  -------
  0  Li+:0.56  0.3148   0.982    0.3148
  1  Li+:0.56  0.982    0.6852   0.6852
  2  Li+:0.56  0.6852   0.3148   0.018
  3  Li+:0.56  0.3148   0.6852   0.018
  4  Li+:0.56  0.982    0.3148   0.3148
  5  Li+:0.56  0.6852   0.982    0.6852
  6  Li+:0.56  0.3148   0.018    0.6852
  7  Li+:0.56  0.6852   0.6852   0.982
  8  Li+:0.56  0.3148   0.3148   0.982
  9  Li+:0.56  0.6852   0.018    0.3148
 10  Li+:0.56  0.018    0.6852   0.3148
 11  Li+:0.56  0.018    0.3148   0.6852
 12  Li+:0.56  0.3148   0.482    0.8148
 13  Li+:0.56  0.982    0.1852   0.1852
 14  Li+:0.56  0.6852   0.8148   0.518
 15  Li+:0.56  0.3148   0.1852   0.518
 16  Li+:0.56  0.982    0.8148   0.8148
 17  Li+:0.56  0.6852   0.482    0.1852
 18

From the above, we see that the reported experimental structure has Li disorder. The occupancy of Li is higher than what would be expected for a Li6PS5Cl nominal composition. We will first adjust the composition by setting the Li occupancies to 0.5 to obtain stoichiometric charge-balanced Li6PS5Cl.

In [3]:
# loop over all sites in the structure
for i, site in enumerate(structure):
    # change the occupancy of Li+ disordered sites to 0.5
    if not site.is_ordered:
        structure[i] = {"Li+": 0.5}
print("The composition after adjustments is %s." % structure.composition.reduced_formula)

The composition after adjustments is Li6PS5Cl.


To keep the number of orderings manageable, we will perform enumeration only on the primitive cell. The primitive cell can be obtained using the *SpacegroupAnalyzer*.

In [4]:
analyzer = SpacegroupAnalyzer(structure)
prim_cell = analyzer.find_primitive()
print(prim_cell)

Full Formula (Li6 P1 S5 Cl1)
Reduced Formula: Li6PS5Cl
abc   :   6.971366   6.971366   6.971366
angles:  60.000000  60.000000  60.000000
pbc   :       True       True       True
Sites (19)
  #  SP             a        b        c
---  -------  -------  -------  -------
  0  Li+:0.5  0.018    0.3524   0.018
  1  Li+:0.5  0.018    0.018    0.6116
  2  Li+:0.5  0.018    0.6116   0.3524
  3  Li+:0.5  0.018    0.3524   0.6116
  4  Li+:0.5  0.018    0.018    0.3524
  5  Li+:0.5  0.018    0.6116   0.018
  6  Li+:0.5  0.3524   0.018    0.6116
  7  Li+:0.5  0.6116   0.018    0.018
  8  Li+:0.5  0.3524   0.018    0.018
  9  Li+:0.5  0.6116   0.018    0.3524
 10  Li+:0.5  0.6116   0.3524   0.018
 11  Li+:0.5  0.3524   0.6116   0.018
 12  P5+      0.5      0.5      0.5
 13  S2-      0.25     0.25     0.25
 14  S2-      0.61947  0.61947  0.14159
 15  S2-      0.61947  0.14159  0.61947
 16  S2-      0.61947  0.61947  0.61947
 17  S2-      0.14159  0.61947  0.61947
 18  Cl-      0        0        0


# Enumerate structures using enumlib

We will use the *EnumerateStructureTransformation* class to enumerate all symmetrically distinct structures. *EnumerateStructureTransformation* is a user-friendly wrapper around enumlib, a fortran library to generate derivative structures written by Hart et al.

In [6]:
enum = EnumerateStructureTransformation(max_cell_size=4)
enumerated = enum.apply_transformation(prim_cell, 100)  # return no more than 100 structures
structures = [d["structure"] for d in enumerated]  
print("%d structures returned." % len(structures))

OSError: [WinError 193] %1 is not a valid Win32 application

# VASP input generation

Pymatgen has useful classes for batch generating VASP input files that use parameters that are compatible with those used in the Materials Project. These parameters have been well-tested over a large database of structures in different chemistries. Using the same parameters also allow the energies computed to be compared with those in the Materials Project database for phase stability and other analyses.

In [6]:
batch_write_input(structures, vasp_input_set=MPRelaxSet, output_dir="Li6PS5Cl_orderings")